# Tutorial 5: Termination and Recursion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-normalization/blob/main/notebooks/05-termination-and-recursion.ipynb)

> "General recursion is the enemy of termination. Structural recursion is its friend." — Varen Tholl

## The Recursion Problem

Recursion is essential for programming, but **general recursion** breaks normalization:

```
loop : A
loop = loop
```

This "defines" a term of any type that never terminates. If allowed, it would let us "prove" any proposition, including False.

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load termination arguments
term_df = pd.read_csv(BASE_URL + "termination_arguments.csv")

# Which systems allow general recursion?
term_df[['type_system', 'terminates', 'why_fails']].head(10)

## The Y Combinator

In the untyped lambda calculus, recursion is encoded by the **Y combinator**:

```
Y = λf. (λx. f (x x)) (λx. f (x x))
```

For any function f, `Y f` computes the fixed point: `Y f = f (Y f)`.

But Y is **untypable** in simply typed lambda calculus (and dependent types). Brennis Mund proved this in Year 835 (EV-835-001).

In [ ]:
# Why is Y untypable?
#
# Y = λf. (λx. f (x x)) (λx. f (x x))
#
# For (x x) to be typed, x must have type A such that:
#   x : A
#   x x : B
# This requires A = A → B, which means A = A → A → A → ...
#
# No finite type satisfies this!

print("Why Y is untypable:")
print("")
print("  Y = λf. (λx. f (x x)) (λx. f (x x))")
print("")
print("  For (x x) to type-check, x needs type A where:")
print("    x : A")
print("    x applied to x needs A = A → B")
print("    So A = A → B = (A → B) → B = ((A → B) → B) → B = ...")
print("")
print("  Infinite type! Not constructible.")

## Structural Recursion

The solution: allow only **structural recursion**, where recursive calls receive *structurally smaller* arguments.

For natural numbers:
```
rec_nat : (n : Nat) → P zero → (∀k. P k → P (succ k)) → P n
rec_nat zero    base step = base
rec_nat (succ n) base step = step n (rec_nat n base step)
```

The recursive call is on `n`, which is smaller than `succ n`.

In [ ]:
# Structural recursion examples
examples = pd.DataFrame({
    'function': ['add m n', 'mult m n', 'factorial n', 'length xs'],
    'recursive_call': ['add m (pred n)', 'add m (mult m (pred n))', 'mult n (factorial (pred n))', 'length (tail xs)'],
    'decreasing_arg': ['n', 'n', 'n', 'xs'],
    'why_smaller': ['pred n < n', 'pred n < n', 'pred n < n', 'tail xs < xs'],
    'terminates': [True, True, True, True]
})

examples

## Positivity for Inductive Types

In Year 948, Tholl discovered that inductive types must satisfy a **positivity condition** (EV-948-003).

An inductive type T is *strictly positive* if T only appears in positive positions in its constructors:

- **Positive**: T appears as the result of an arrow, or not at all
- **Negative**: T appears as an argument of an arrow

Bad example:
```
data Bad = MkBad (Bad → Nat)
```

This allows encoding Omega!

In [ ]:
# Positivity check
positivity = pd.DataFrame({
    'inductive_type': [
        'data Nat = Zero | Succ Nat',
        'data List A = Nil | Cons A (List A)',
        'data Tree A = Leaf A | Node (Tree A) (Tree A)',
        'data Bad = MkBad (Bad -> Nat)',
        'data Bad2 = MkBad2 ((Bad2 -> Nat) -> Nat)'
    ],
    'strictly_positive': [True, True, True, False, True],
    'occurrences_of_self': [
        'Positive (result of Succ)',
        'Positive (result of Cons)',
        'Positive (result of Node)',
        'Negative (argument of arrow)',
        'Positive (double negation = positive)'
    ]
})

positivity

## Why Negativity Breaks Normalization

With a negative inductive type, we can encode Omega:

```
data Bad = MkBad (Bad → Nat)

-- Extract the function
unBad : Bad → (Bad → Nat)
unBad (MkBad f) = f

-- Self-application
omega : Nat
omega = unBad (MkBad unBad) (MkBad unBad)
```

This term has type Nat but doesn't terminate!

In [ ]:
# The encoding of Omega via negative types
print("data Bad = MkBad (Bad → Nat)")
print("")
print("unBad : Bad → (Bad → Nat)")
print("unBad (MkBad f) = f")
print("")
print("omega = unBad (MkBad unBad) (MkBad unBad)")
print("")
print("Reduction:")
print("  = unBad (MkBad unBad) (MkBad unBad)")
print("  = unBad (MkBad unBad)       -- (MkBad unBad) ")
print("  = unBad (MkBad unBad) (MkBad unBad)  -- same as before!")
print("")
print("This loops forever, but has type Nat!")

## Universe Levels

Another threat to normalization: `Type : Type`.

If types contain themselves, we can encode Russell's paradox:

```
R = { x : Type | x ∉ x }
```

Is R ∈ R? This leads to contradiction.

Tholl confirmed in Year 950 (EV-950-002) that Linn's universe hierarchy is essential:

```
Type₀ : Type₁ : Type₂ : ...
```

In [ ]:
# Universe stratification
universes = pd.DataFrame({
    'universe': ['Type₀', 'Type₁', 'Type₂', 'Type₃'],
    'contains': ['Nat, Bool, ...', 'Type₀, Nat → Type₀, ...', 'Type₁, Type₀ → Type₁, ...', 'Type₂, ...'],
    'type_of': ['Type₁', 'Type₂', 'Type₃', 'Type₄']
})

universes

## The Termination Checker

Tholl formalized the **termination checker** in Year 968 (EV-968-005):

1. Every recursive function must have a *decreasing argument*
2. Recursive calls must be on *structurally smaller* subterms
3. The checker syntactically verifies this property

In [ ]:
# Termination checking
check_examples = pd.DataFrame({
    'function': [
        'add m zero = m\nadd m (succ n) = succ (add m n)',
        'f n = f n',
        'g n = g (succ n)',
        'h n = h (n - 1)'
    ],
    'decreasing_arg': ['n', 'none', 'n increases!', 'maybe...'],
    'passes_check': [True, False, False, False],
    'reason': [
        'n → succ n → ... → zero',
        'No structural decrease',
        'n gets bigger!',
        'subtraction not structural'
    ]
})

check_examples

## Summary: Requirements for Normalization

| Requirement | What It Prevents | How Enforced |
|-------------|------------------|---------------|
| No general recursion | Infinite loops | Structural recursion only |
| Strict positivity | Encoding Omega | Syntactic check on inductives |
| Universe stratification | Russell's paradox | Type levels |
| Termination checking | Non-obvious loops | Syntactic decrease check |

Together, these constraints ensure that every well-typed term terminates.

---

**Next Tutorial:** Consistency from Normalization — How termination proves logical soundness